# Линейные классификаторы и SVM

## Линейные классификаторы
Линейные классификаторы - это семейство алгоритмов, которые классифицируют данные на основе значений линейной комбинации признаков. Если результат этой линейной комбинации превышает определенный порог, объект присваивается одному классу, в противном случае - другому.

Примеры линейных классификаторов:
- Логистическая регрессия
- Перцептрон

## SVM (Support Vector Machines)
SVM - это алгоритм, который ищет гиперплоскость, максимизирующую разницу между двумя классами. SVM хорошо работает, когда классы хорошо разделимы. Он также может работать с нелинейно разделимыми данными с использованием так называемых ядер (kernels).

Основная идея SVM - найти гиперплоскость, которая наилучшим образом разделяет данные на классы. 'Наилучшим образом' означает, что расстояние от гиперплоскости до ближайших точек (или 'опорных векторов') каждого класса максимально.

In [ ]:
import pandas as pd

# Загрузка данных
wine_data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine_data = pd.read_csv(wine_data_url, sep=';')
wine_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# Разделение данных на обучающую и тестовую выборки
X = wine_data.drop('quality', axis=1)
y = wine_data['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Инициализация моделей
logistic_regression = LogisticRegression(max_iter=10000, random_state=42)
perceptron = Perceptron(max_iter=10000, random_state=42)
svm = SVC(kernel='linear', random_state=42)

# Обучение моделей
logistic_regression.fit(X_train_scaled, y_train)
perceptron.fit(X_train_scaled, y_train)
svm.fit(X_train_scaled, y_train)

# Возвращаем обученные модели
logistic_regression, perceptron, svm

(LogisticRegression(max_iter=10000, random_state=42),
 Perceptron(max_iter=10000, random_state=42),
 SVC(kernel='linear', random_state=42))

## Метрики классификации для многоклассовой задачи

Метрики классификации используются для оценки производительности модели классификации. В контексте многоклассовой классификации некоторые из основных метрик включают:

- **Accuracy (Точность)**: Это отношение правильно предсказанных наблюдений ко всем наблюдениям.

- **Precision (Точность), Recall (Полнота) и F1-Score**: Эти метрики могут быть адаптированы для многоклассовой классификации путем усреднения результатов для каждого класса. Существует несколько стратегий усреднения, таких как "macro" (усреднение метрики для каждого класса без учета пропорции для каждого класса), "weighted" (учитывает пропорцию для каждого класса) и "micro" (вычисляет общее количество истинных положительных, ложных положительных и ложных отрицательных результатов).

- **Confusion Matrix (Матрица ошибок)**: Это таблица, которая показывает, сколько примеров каждого класса было правильно и неправильно классифицировано другими классами.

- **Classification Report**: Это комплексный отчет, который показывает метрики `precision`, `recall` и `f1-score` для каждого класса.

Важно выбрать правильную метрику в зависимости от задачи и данных. Например, в случае несбалансированных классов, точность может быть не лучшей метрикой, и стоит обратить внимание на F1-Score или другие метрики, которые учитывают баланс классов.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Предсказания моделей
lr_predictions = logistic_regression.predict(X_test_scaled)
perceptron_predictions = perceptron.predict(X_test_scaled)
svm_predictions = svm.predict(X_test_scaled)

# Вычисление метрик
metrics_data = {
    'Model': ['Logistic Regression', 'Perceptron', 'SVM'],
    'Accuracy': [
        accuracy_score(y_test, lr_predictions),
        accuracy_score(y_test, perceptron_predictions),
        accuracy_score(y_test, svm_predictions)
    ],
    'Precision': [
        precision_score(y_test, lr_predictions, average='weighted', zero_division=0),
        precision_score(y_test, perceptron_predictions, average='weighted', zero_division=0),
        precision_score(y_test, svm_predictions, average='weighted', zero_division=0)
    ],
    'Recall': [
        recall_score(y_test, lr_predictions, average='weighted'),
        recall_score(y_test, perceptron_predictions, average='weighted'),
        recall_score(y_test, svm_predictions, average='weighted')
    ],
    'F1-Score': [
        f1_score(y_test, lr_predictions, average='weighted'),
        f1_score(y_test, perceptron_predictions, average='weighted'),
        f1_score(y_test, svm_predictions, average='weighted')
    ]
}

# Создание DataFrame для отображения метрик
metrics_df = pd.DataFrame(metrics_data)
metrics_df

## Выбор наиболее показательных метрик для данных

При выборе метрик для оценки производительности модели классификации важно учитывать следующие аспекты:

- **Баланс классов**: Если классы в данных несбалансированы, то простая метрика, такая как точность (accuracy), может быть неинформативной. В таком случае стоит обратить внимание на другие метрики, такие как F1-Score.

- **Цель задачи**: В зависимости от цели задачи, одна метрика может быть более важной, чем другая. Например, в медицинских тестах высокая полнота (recall) может быть критически важной, чтобы не пропустить больных.

Для нашего набора данных о качестве вина важно учитывать, что классы могут быть несбалансированы, так как некоторые оценки качества могут встречаться реже других. Поэтому, помимо общей точности, стоит также учитывать F1-Score, который учитывает баланс между точностью (precision) и полнотой (recall).

## Интерпретация результатов - выбор лучшей модели

Исходя из вычисленных метрик, мы можем сделать следующие выводы:

- **Logistic Regression (Логистическая регрессия)**: Эта модель показала наилучшие результаты по всем метрикам среди рассмотренных. Она достигла наивысшей точности, точности (precision), полноты (recall) и F1-Score.

- **Perceptron (Перцептрон)**: Эта модель показала наименьшую точность и F1-Score среди всех моделей. Она может быть менее стабильной из-за своей природы, особенно на данных, которые не являются линейно разделимыми.

- **SVM**: SVM с линейным ядром показал результаты, которые лучше, чем у перцептрона, но хуже, чем у логистической регрессии. Он может быть более устойчивым к выбросам, чем другие линейные модели.

Исходя из этих результатов, **логистическая регрессия** является наилучшей моделью для нашего набора данных. Однако стоит учитывать, что выбор модели также зависит от конкретной задачи и требований к производительности.

## Кросс-валидация

Кросс-валидация - это метод оценки производительности модели путем разделения данных на несколько подмножеств (или 'фолдов'). Модель обучается на нескольких из этих подмножеств и тестируется на оставшихся подмножествах. Этот процесс повторяется несколько раз, и каждый раз разные подмножества используются в качестве обучающих и тестовых данных.

Основные преимущества кросс-валидации:

- **Более надежная оценка производительности**: Поскольку модель тестируется на разных подмножествах данных, результаты обычно более надежны и стабильны.

- **Использование всего набора данных**: Все данные используются для обучения и тестирования, что может быть особенно полезно при ограниченном объеме данных.

Однако кросс-валидация также требует больше времени на обучение, так как модель обучается несколько раз.

In [ ]:
from sklearn.model_selection import cross_val_score

# Количество фолдов для кросс-валидации
cv = 5

# Метрика для оценки
scoring = 'f1_weighted'

# Вычисление метрик с использованием кросс-валидации
lr_scores = cross_val_score(logistic_regression, X_train_scaled, y_train, cv=cv, scoring=scoring)
perceptron_scores = cross_val_score(perceptron, X_train_scaled, y_train, cv=cv, scoring=scoring)
svm_scores = cross_val_score(svm, X_train_scaled, y_train, cv=cv, scoring=scoring)

lr_scores, perceptron_scores, svm_scores

(array([0.5679372 , 0.56946536, 0.54671045, 0.59562506, 0.65026839]),
 array([0.45733247, 0.45220759, 0.39682424, 0.47602163, 0.5586765 ]),
 array([0.51103999, 0.49569587, 0.53067349, 0.54778145, 0.58732498]))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(lr_scores, '-o', label='Logistic Regression')
plt.plot(perceptron_scores, '-o', label='Perceptron')
plt.plot(svm_scores, '-o', label='SVM')
plt.title('Cross-Validation Accuracy Scores')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.xticks(range(cv))
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Вычисление усредненных метрик
average_scores = {
    'Model': ['Logistic Regression', 'Perceptron', 'SVM'],
    'Average W-F1': [
        lr_scores.mean(),
        perceptron_scores.mean(),
        svm_scores.mean()
    ]
}

# Создание DataFrame для отображения усредненных метрик
average_scores_df = pd.DataFrame(average_scores)
average_scores_df

### Задание - Используйте метод GridSearchCV из sklearn.model_selection для того, чтобы улучшить результаты SVM и обойти логистическую регрессию. 

In [1]:
#

## Стратифицированный k-fold

Стратифицированный k-fold кросс-валидация является вариацией k-fold кросс-валидации, которая может обеспечить более точную оценку производительности модели, особенно когда у вас есть несбалансированные классы.

В обычной k-fold кросс-валидации датасет разбивается на 'k' подмножеств (или 'фолдов') случайным образом. Однако, это может привести к ситуации, когда в одном из фолдов может быть непропорционально много примеров одного класса по сравнению с другими классами.

Стратифицированный k-fold решает эту проблему, обеспечивая каждый фолд равномерным распределением классов. То есть, в каждом фолде будет примерно одинаковое количество примеров каждого класса. Это особенно полезно, когда у вас есть несбалансированные классы, так как это обеспечивает более точную и надежную оценку производительности модели.

Для использования стратифицированного k-fold в scikit-learn, можно использовать класс `StratifiedKFold`.

In [ ]:
from sklearn.model_selection import StratifiedKFold

# Инициализация стратифицированного k-fold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Вычисление метрик с использованием стратифицированного k-fold кросс-валидации
lr_scores_stratified = cross_val_score(logistic_regression, X_train_scaled, y_train, cv=stratified_kfold, scoring='accuracy')
perceptron_scores_stratified = cross_val_score(perceptron, X_train_scaled, y_train, cv=stratified_kfold, scoring='accuracy')
svm_scores_stratified = cross_val_score(svm, X_train_scaled, y_train, cv=stratified_kfold, scoring='accuracy')

lr_scores_stratified, perceptron_scores_stratified, svm_scores_stratified

(array([0.59765625, 0.59375   , 0.640625  , 0.58984375, 0.60392157]),
 array([0.453125  , 0.47265625, 0.53125   , 0.484375  , 0.43921569]),
 array([0.58984375, 0.60546875, 0.6015625 , 0.57421875, 0.56470588]))

In [ ]:
# Вычисление усредненных метрик
average_scores = {
    'Model': ['Logistic Regression', 'Perceptron', 'SVM'],
    'Average W-F1': [
        lr_scores_stratified.mean(),
        perceptron_scores_stratified.mean(),
        svm_scores_stratified.mean()
    ]
}

# Создание DataFrame для отображения усредненных метрик
average_scores_df = pd.DataFrame(average_scores)
average_scores_df